In [1]:
!pip install -U sentence-transformers

In [2]:
from sentence_transformers import SentenceTransformer, InputExample, losses, util
from torch.utils.data import DataLoader
import pandas as pd
from scipy.stats import pearsonr
import torch

In [3]:
from google.colab import drive
drive.mount('/content/drive/')

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).


In [4]:
validationData = pd.read_csv('/content/drive/My Drive/Assignment 3 NLP/dev.csv', delimiter="\t")
trainingData = pd.read_csv('/content/drive/My Drive/Assignment 3 NLP/train.csv', delimiter="\t")

In [5]:
null_mask = validationData.isnull().any(axis=1)
validationData[null_mask]

,score,setence1,sentence2
764,1.4,The grass family is one of the most widely dis...,NaN
1024,3.8,"She was crying and scared,' said Isa Yasin, th...",NaN


In [6]:
validationData.isnull().sum().sum()
trainingData.isnull().sum().sum()

3

In [7]:
validationData

,score,setence1,sentence2
0,5.00,A man with a hard hat is dancing.,A man wearing a hard hat is dancing.
1,4.75,A young child is riding a horse.,A child is riding a horse.
2,5.00,A man is feeding a mouse to a snake.,The man is feeding a mouse to the snake.
3,2.40,A woman is playing the guitar.,A man is playing guitar.
4,2.75,A woman is playing the flute.,A man is playing a flute.
...,...,...,...
1465,2.00,Scientists prove there is water on Mars,Has Nasa discovered water on Mars?
1466,0.00,Pranab stresses need to strive for peace by na...,WTO: India regrets action of developed nations
1467,2.00,Volkswagen skids into red in wake of pollution...,"Volkswagen's ""gesture of goodwill"" to diesel o..."
1468,0.00,Obama is right: Africa deserves better leadership,Obama waiting for midterm to name attorney gen...


In [8]:
validationData = validationData.dropna(subset=['setence1', 'sentence2', 'score'])
validationData.reset_index(drop=True, inplace = True)
trainingData = trainingData.dropna(subset=['sentence1', 'sentence2', 'score'])
trainingData.reset_index(drop=True, inplace = True)
trainingData

,score,sentence1,sentence2
0,5.00,A plane is taking off.,An air plane is taking off.
1,3.80,A man is playing a large flute.,A man is playing a flute.
2,3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,2.60,Three men are playing chess.,Two men are playing chess.
4,4.25,A man is playing the cello.,A man seated is playing the cello.
...,...,...,...
5703,0.00,Severe Gales As Storm Clodagh Hits Britain,Merkel pledges NATO solidarity with Latvia
5704,0.00,Dozens of Egyptians hostages taken by Libyan t...,Egyptian boat crash death toll rises as more b...
5705,0.00,President heading to Bahrain,President Xi: China to continue help to fight ...
5706,0.00,"China, India vow to further bilateral ties",China Scrambles to Reassure Jittery Stock Traders


In [9]:
updatedLabels = []
maxLabelScore = max(trainingData['score'])
minLabelScore = min(trainingData['score'])
model = SentenceTransformer("distilbert-base-nli-mean-tokens")
train_examples = []
for rowNo in range(len(trainingData)):
  sentence1 = trainingData.iloc[rowNo]['sentence1']
  sentence2 = trainingData.iloc[rowNo]['sentence2']
  score = float(trainingData.iloc[rowNo]['score'])
  updatedScore = 1*((score - minLabelScore)) / (maxLabelScore - minLabelScore)
  textsList = [sentence1, sentence2]
  train_examples.append(InputExample(texts=textsList, label = updatedScore))

train_batch_size = 1
train_dataloader = DataLoader(train_examples, shuffle=True, batch_size=train_batch_size)
train_loss = losses.CosineSimilarityLoss(model=model)

model.fit(
    [(train_dataloader, train_loss)],
    epochs=3,
)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Epoch:   0%|          | 0/3 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5708 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5708 [00:00<?, ?it/s]

Iteration:   0%|          | 0/5708 [00:00<?, ?it/s]

In [13]:
cosineScores = []
for rowNo in range(len(validationData)):
  sentence1 = validationData.iloc[rowNo]['setence1']
  sentence2 = validationData.iloc[rowNo]['sentence2']
  embeddingSen1 = model.encode(sentence1, convert_to_tensor= True)
  embeddingSen2 = model.encode(sentence2, convert_to_tensor= True)
  cosineScore = util.cos_sim(embeddingSen1, embeddingSen2)
  cosineScores.append(cosineScore[0][0].item())
cosineScores

[0.9857930541038513,
 0.983822226524353,
 0.995445728302002,
 0.5627259612083435,
 0.8306836485862732,
 0.7068390846252441,
 0.9910851716995239,
 0.7493892908096313,
 0.8702540993690491,
 0.9581118226051331,
 0.7069453597068787,
 0.3976115584373474,
 0.9855996370315552,
 0.9791276454925537,
 0.9915573596954346,
 0.18078942596912384,
 0.9881919026374817,
 0.9822873473167419,
 0.9077316522598267,
 0.4429203271865845,
 0.6434526443481445,
 0.4521556496620178,
 0.5774319171905518,
 0.3854381740093231,
 0.04138494282960892,
 0.39980995655059814,
 0.7996740341186523,
 0.5723130106925964,
 0.17048121988773346,
 -0.17596738040447235,
 0.582896888256073,
 0.9597194194793701,
 0.9465070962905884,
 0.9749370813369751,
 0.9127289056777954,
 0.8440260887145996,
 0.991021454334259,
 0.9802122712135315,
 0.9155527949333191,
 0.48439449071884155,
 0.8832654356956482,
 0.6774161458015442,
 0.39910104870796204,
 0.746152400970459,
 0.4048132598400116,
 0.16236044466495514,
 0.03688817471265793,
 0.01999

In [14]:
updatedLabels = []
for labelVal in cosineScores:
  newLabelVal = 4*((labelVal - min(cosineScores)) / (max(cosineScores) - min(cosineScores))) + 1
  updatedLabels.append(newLabelVal)
updatedLabels

[4.951675714367988,
 4.944972033254587,
 4.984508850249123,
 3.5126320812019234,
 4.424078068031136,
 4.0028263699834135,
 4.969676613657736,
 4.147558977004374,
 4.558675171461559,
 4.857519254044412,
 4.003187859879835,
 2.951002888003707,
 4.95101781492105,
 4.929003627633728,
 4.971282739826808,
 2.2134921846246796,
 4.959835289450098,
 4.939751210863806,
 4.686153377116845,
 3.1051186211583226,
 3.7872202525576126,
 3.136532153980149,
 3.562653727132783,
 2.909595669977515,
 1.7393141233537386,
 2.958480640022626,
 4.31860026317245,
 3.5452419975788745,
 2.178429286611829,
 1.0,
 3.581242579703308,
 4.862987422298141,
 4.818046300940718,
 4.914749612682608,
 4.703151309206595,
 4.469461776289226,
 4.969459881913892,
 4.932692933099343,
 4.712756641131881,
 3.2461911176351963,
 4.602932645073196,
 3.902745570757231,
 2.956069321972657,
 4.136548842223053,
 2.975499150477202,
 2.1508068359959447,
 1.7240185694399752,
 1.6665692235475036,
 3.4070816945244378,
 4.144180070322507,
 2.5

In [15]:
coeff, _ = pearsonr(validationData['score'], updatedLabels)
coeff

0.8646000950034725

In [17]:
checkpoint = {
    'epoch': 3,
    'model_state_dict': model.state_dict(),
}

checkpoint_path = '/content/drive/My Drive/Assignment 3 NLP/Task1 Part C checkpoint.pth'

torch.save(checkpoint, checkpoint_path)